# Coordinator of a MuSig2 Protocol.

This notebook demonstrates how to use the Coordinator class to establish an 2-of-2 musig2 cohort for a btc1 SMTAggregate beacon and coordinate the signing of a Beacon signal with all cohort participants.

The coordinator class makes use of the python didcomm-messaging library for the packaging and encryption of messages. These encrypted messages are then sent over websocket to an endpoint defined in the cohort participants DID documents.

Initially, this example uses did:peer as the DID method against which participants and coordinators identify themselves too each other. Furthermore, the didcomm messaging library currently requires that Ed25519 keys are used to encrypt the symmetric key that is used to encrypt each message. 

In the future we would replace did:peer with did:btc1 and Ed25519 keys with secp256k1. This appears possible without too much additional lift.

In [19]:
import asyncio
from musig2_protocols.beacon_coordinator import BeaconCoordinator
from buidl.hd import HDPrivateKey, secure_mnemonic
import json

# Generate a HD Key

Actually, the coordinator does not need a HDKey in the example as it is currently developed. 

In [7]:
secure_mnemonic = secure_mnemonic()
hd_priv = HDPrivateKey.from_mnemonic(secure_mnemonic)

# Define DIDComm Messaging host and port

In [8]:
didcomm_host = "localhost"
didcomm_port = 8765

# Instantiate a Coordinator

This creates a BeaconCoordinator and registers the appropriate default handlers for handling the messages of the musig2 keygen and signing protocols.

In [12]:
# Name is for logging purposes
name = "Beacon Coordinator"
coordinator = await BeaconCoordinator.create(
        name="Coordinator", 
        port=didcomm_port,
        host=didcomm_host
    )

Coordinator: Registering handler for message type https://btc1.tools/musig2/keygen/subscribe
Coordinator: Registering handler for message type https://btc1.tools/musig2/keygen/opt_in
Coordinator: Registering handler for message type https://btc1.tools/musig2/sign/request_signature
Coordinator: Registering handler for message type https://btc1.tools/musig2/sign/nonce_contribution
Coordinator: Registering handler for message type https://btc1.tools/musig2/sign/signature_authorization


In [16]:
# A did:peer is generated for the coordinator
# Future iterations will make this one DID per cohort participant
# With a public DID for subscribing 
did = coordinator.did
did

'did:peer:2.Vz6MkgExXQ5es6n7Fie6SdyEdhYgYA5VDtzDwCJUGWGspWemo.Ez6LSePN8SKTBYuyMjpYZrYjFqoKARsnrPqYwBuRSByoLX8bP.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ'

## Resolve the DID

The DID can be resolved to its DID document, which contains a verificationMethod that can be used for Key Agreement (#key-2) and a DIDCommMessaging service with a service endpoint identifying the websocket that participants should send DIDComm messages to.

In [17]:
from didcomm_messaging.resolver.peer import Peer2, Peer4
from didcomm_messaging.resolver import PrefixResolver
resolver = PrefixResolver({"did:peer:2": Peer2(), "did:peer:4": Peer4()})


In [20]:
did_document = await resolver.resolve(did)
print(json.dumps(did_document, indent=2))

{
  "@context": [
    "https://www.w3.org/ns/did/v1",
    "https://w3id.org/security/multikey/v1"
  ],
  "id": "did:peer:2.Vz6MkgExXQ5es6n7Fie6SdyEdhYgYA5VDtzDwCJUGWGspWemo.Ez6LSePN8SKTBYuyMjpYZrYjFqoKARsnrPqYwBuRSByoLX8bP.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ",
  "verificationMethod": [
    {
      "type": "Multikey",
      "id": "#key-1",
      "controller": "did:peer:2.Vz6MkgExXQ5es6n7Fie6SdyEdhYgYA5VDtzDwCJUGWGspWemo.Ez6LSePN8SKTBYuyMjpYZrYjFqoKARsnrPqYwBuRSByoLX8bP.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ",
      "publicKeyMultibase": "z6MkgExXQ5es6n7Fie6SdyEdhYgYA5VDtzDwCJUGWGspWemo"
    },
    {
      "type": "Multikey",
      "id": "#key-2",
      "controller": "did:peer:2.Vz6MkgExXQ5es6n7Fie6SdyEdhYgYA5VDtzDwCJUGWGspWemo.Ez6LSePN8SKTBYuyMjpYZrYjFqoKARsnrPqYwBuRSByoLX8bP.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ",

## Start Coordinator 

This starts up the websocket defined in the DID document so that the coordinator is ready to receive messages

In [22]:
task = asyncio.create_task(coordinator.start())

Coordinator: Starting websocket server on ws://localhost:8765
Coordinator: WebSocket server started successfully
Coordinator: New client connected
Coordinator: Received raw message
Coordinator: Successfully unpacked message: {
  "type": "https://btc1.tools/musig2/keygen/subscribe",
  "id": "d6305d82-872c-4641-96bb-fb556715a1fd",
  "to": "did:peer:2.Vz6MkgExXQ5es6n7Fie6SdyEdhYgYA5VDtzDwCJUGWGspWemo.Ez6LSePN8SKTBYuyMjpYZrYjFqoKARsnrPqYwBuRSByoLX8bP.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ",
  "from": "did:peer:2.Vz6MkrW4B8Pyv89nQ3EdN3zCGrLEgEZ5VqsSXjqjhjuwjSzFc.Ez6LSboVcakMcH613b8S6jFgEuA4byKMZSndk7yTUtp928DL2.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjYiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ",
  "body": {}
}
Routing - https://btc1.tools/musig2/keygen/subscribe
Coordinator: Preparing to send message to did:peer:2.Vz6MkrW4B8Pyv89nQ3EdN3zCGrLEgEZ5VqsSXjqjhjuwjSzFc.Ez6LSboVcakMcH613b8S6jFgEuA4byKMZSndk7yTUtp928DL2.SeyJ0IjoiZ

# Coordinator shares DID and awaits potential cohort participants to subscribe

This is an optional feature. Coordinators need some way to announce to potential cohort participants that they are willing to coordinator beacon cohorts and will be advertising these cohorts. 

In this case, the coordinator is sharing a DID, maybe on a website or through a DM, interested parties then send a subscribe message which the coordinator accepts and adds the subscriber DID into their list of potential cohort participants. When the coordinator advertises a new cohort, they can send this advertisement to each subscriber.

There could be many alternative approaches through which cohort participants find out about CohortAdverts from BeaconCoordinators.

In [26]:
print("Copy this across to Fred and Lucia notebooks, which will act as participants in the Beacon cohort \n")
print(f"coordinator_did = '{did}'")

Copy this across to Fred and Lucia notebooks, which will act as participants in the Beacon cohort 

coordinator_did = 'did:peer:2.Vz6MkgExXQ5es6n7Fie6SdyEdhYgYA5VDtzDwCJUGWGspWemo.Ez6LSePN8SKTBYuyMjpYZrYjFqoKARsnrPqYwBuRSByoLX8bP.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ'


# Create Coordinator DID

In [3]:
secrets = InMemorySecretsManager()
crypto = AskarCryptoService()

# verkey = Key.generate(KeyAlg.ED25519)
# verkey_sk_bytes = verkey.get_secret_bytes()
verkey_sk_bytes = b'\xc4\xce4LB/\xf4\xfa-\x1e\x9f\xe7NqT\x9ao\xf1\xb8xI[}`~\xe0\xdfcY\x83<\xae'
verkey = Key.from_secret_bytes(KeyAlg.ED25519, verkey_sk_bytes)


# xkey = Key.generate(KeyAlg.X25519)
# xkey_sk_bytes = xkey.get_secret_bytes()
xkey_sk_bytes = b'\xe9!\xae,\xa8F\xc2y\t\xc6\x86\xcd1\xd4Ys|\xe4\xab\xa8\xe0xx\xe9\xff\x11&\xb3\xbe\x11\xa8\xef'
xkey = Key.from_secret_bytes(KeyAlg.X25519,xkey_sk_bytes)


In [4]:

did = generate(
    [
        KeySpec.verification(
            multibase.encode(
                multicodec.wrap("ed25519-pub", verkey.get_public_bytes()),
                "base58btc",
            )
        ),
        KeySpec.key_agreement(
            multibase.encode(
                multicodec.wrap("x25519-pub", xkey.get_public_bytes()), "base58btc"
            )
        ),
    ],
    [
                {
                    "type": "DIDCommMessaging",
                    "serviceEndpoint": {
                        "uri": didcomm_messaging_uri,
                        "accept": ["didcomm/v2"],
                        "routingKeys": [],
                    },
                },
    ],
)
await secrets.add_secret(AskarSecretKey(verkey, f"{did}#key-1"))
await secrets.add_secret(AskarSecretKey(xkey, f"{did}#key-2"))


In [5]:
did

'did:peer:2.Vz6MksvTpc5zbWrfdZCo3NyjABQQUDqC7yksyBk7PYmCqQ74i.Ez6LSgRMYHNDvqr93XiBp5qNtF1tzQ1cm3gKaoeYukcrnEB1p.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ'

In [6]:
resolver = PrefixResolver({"did:peer:2": Peer2(), "did:peer:4": Peer4()})

did_document = await resolver.resolve(did)
print(json.dumps(did_document, indent=2))

{
  "@context": [
    "https://www.w3.org/ns/did/v1",
    "https://w3id.org/security/multikey/v1"
  ],
  "id": "did:peer:2.Vz6MksvTpc5zbWrfdZCo3NyjABQQUDqC7yksyBk7PYmCqQ74i.Ez6LSgRMYHNDvqr93XiBp5qNtF1tzQ1cm3gKaoeYukcrnEB1p.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ",
  "verificationMethod": [
    {
      "type": "Multikey",
      "id": "#key-1",
      "controller": "did:peer:2.Vz6MksvTpc5zbWrfdZCo3NyjABQQUDqC7yksyBk7PYmCqQ74i.Ez6LSgRMYHNDvqr93XiBp5qNtF1tzQ1cm3gKaoeYukcrnEB1p.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ",
      "publicKeyMultibase": "z6MksvTpc5zbWrfdZCo3NyjABQQUDqC7yksyBk7PYmCqQ74i"
    },
    {
      "type": "Multikey",
      "id": "#key-2",
      "controller": "did:peer:2.Vz6MksvTpc5zbWrfdZCo3NyjABQQUDqC7yksyBk7PYmCqQ74i.Ez6LSgRMYHNDvqr93XiBp5qNtF1tzQ1cm3gKaoeYukcrnEB1p.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ",

# Instantiate DIDComm Messaging 

This is used to pack and unpack DIDComm messages.

In [7]:
packaging = PackagingService()
router = RoutingService()

dmp = DIDCommMessaging(
    crypto=crypto,
    secrets=secrets,
    resolver=resolver,
    packaging=packaging,
    routing=router,
)

# Create send message functon

Sends packed DIDComm messages to a websocket endpoint

In [8]:
async def send_message(message, to, frm):
    packy = await dmp.pack(
        message=message,
        to=to,
        frm=frm,
    )
    packed = packy.message            
    # Get the first http endpoint from the last DID in the DID chain
    endpoint = packy.get_endpoint("ws")
    async with websockets.connect(endpoint) as websocket:
        message = "Hello from the client!"
        print(f"Sending message: {packed}")
        await websocket.send(packed)  # Send message to the server
        
        response = await websocket.recv()  # Wait for the server's response
        print(f"Received response: {response}")

# Setup DIDComm Messaging Websocket

This is where DIDComm messages are sent to the coordinator

In [9]:
import asyncio
import websockets

inbox = []


# This handler will be called for every client connection.
async def handle_messages(websocket):
    print("Client connected")
    try:
        async for message in websocket:
            print(f"Received DIDComm Message: \n\n {json.dumps(json.loads(message.decode()),indent=2)} \n\n")
            # Echo the received message back to the client.
            # await websocket.send(f"Echo: {message}")

            unpacked =  await dmp.packaging.unpack(crypto, resolver, secrets,message)
            msg = json.loads(unpacked[0].decode())
            inbox.append(msg)
            print(f"Unpacked Message : \n\n {json.dumps(msg, indent=2)} \n\n")
    except websockets.exceptions.ConnectionClosed as e:
        print("Connection closed", e)

# The main function to start the server.
async def run_websocket():
    # Start the server on localhost at port 8765.
    async with websockets.serve(handle_messages, didcomm_host, didcomm_port):
        print(f"WebSocket server started on {didcomm_messaging_uri}")
        # Run the server until it is manually stopped.
        await asyncio.Future()  # Run forever

In [10]:
asyncio.create_task(run_websocket())

<Task pending name='Task-7' coro=<run_websocket() running at /tmp/ipykernel_9458/3993558428.py:24>>

WebSocket server started on ws://localhost:8765
Client connected
Received DIDComm Message: 

 {
  "protected": "eyJ0eXAiOiAiYXBwbGljYXRpb24vZGlkY29tbStlbmNyeXB0ZWQiLCAiYWxnIjogIkVDREgtMVBVK0EyNTZLVyIsICJlbmMiOiAiQTI1NkNCQy1IUzUxMiIsICJhcHUiOiAiWkdsa09uQmxaWEk2TWk1V2VqWk5hM05yYVhCNWIyOHlSMHR5ZW1GSWFFTk1ka3BHVkRaWlkxVlJjVXcxWnpNM1dWUk5hbTFZU0V4WVYzVjNMa1Y2Tmt4VGJrVklXRm8zT1ZsQ1FtUmpXWFJET1hoRlJtbzFkVkl5U21obGFqRXpNak5ZY1hCT1NuZHhia28zWWpNdVUyVjVTakJKYW05cFdrY3dhVXhEU25wSmFuQTNTVzVXZVdGVFNUWkpibVI2VDJrNGRtSkhPV3BaVjNodllqTk9NRTlxWnpOT2FsbHBURU5LYUVscWNHSkpiVkp3V2tkT2RtSlhNSFprYWtscFdGTjNhV05wU1RaWE1URTVabEVqYTJWNUxUSSIsICJhcHYiOiAiN1B2cUNRS0pvbDQwN3pwLW5jeHBKSTZvZmNoQkxBeGJFVVhYSl9COHJEdyIsICJlcGsiOiB7ImNydiI6ICJYMjU1MTkiLCAia3R5IjogIk9LUCIsICJ4IjogIm5ESUQyaUQzUTNjNDBVOVV1VVlLZUh3aWZ0aC1yenY2NjFPeERYNTVRMGsifSwgInNraWQiOiAiZGlkOnBlZXI6Mi5WejZNa3NraXB5b28yR0tyemFIaENMdkpGVDZZY1VRcUw1ZzM3WVRNam1YSExYV3V3LkV6NkxTbkVIWFo3OVlCQmRjWXRDOXhFRmo1dVIySmhlajEzMjNYcXBOSndxbko3YjMuU2V5SjBJam9pWkcwaUxD

# Coordinator Shares DID with participants

Maybe this happens on their website. Or something like that.

In [12]:
print(f"coordinator_did='{did}'")

coordinator_did='did:peer:2.Vz6MksvTpc5zbWrfdZCo3NyjABQQUDqC7yksyBk7PYmCqQ74i.Ez6LSgRMYHNDvqr93XiBp5qNtF1tzQ1cm3gKaoeYukcrnEB1p.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ'


# Coordinator waits for participants to subscribe

In [27]:
coordinator.subscribers

['did:peer:2.Vz6MkrW4B8Pyv89nQ3EdN3zCGrLEgEZ5VqsSXjqjhjuwjSzFc.Ez6LSboVcakMcH613b8S6jFgEuA4byKMZSndk7yTUtp928DL2.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjYiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ',
 'did:peer:2.Vz6Mkq8t1GaVjQ5zf8PnnahySB2bHukdVVCUpYQiAf2Uxisxx.Ez6LSqKuutdBvc8zyQkSjJZ7zUBbopj7VDMhasaGLEZYSwsxL.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjciLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ']

## Coordinator Advertises a Beacon Cohort

This sends a CohortAdvert message to each subscriber

In [28]:

# TODO: refine the data model of the message. At least include the type of cohort.
await coordinator.announce_new_cohort(min_participants=2)

Creating new cohort and announcing to 2 subscribers
Sending cohort announcement to did:peer:2.Vz6MkrW4B8Pyv89nQ3EdN3zCGrLEgEZ5VqsSXjqjhjuwjSzFc.Ez6LSboVcakMcH613b8S6jFgEuA4byKMZSndk7yTUtp928DL2.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjYiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ
Coordinator: Preparing to send message to did:peer:2.Vz6MkrW4B8Pyv89nQ3EdN3zCGrLEgEZ5VqsSXjqjhjuwjSzFc.Ez6LSboVcakMcH613b8S6jFgEuA4byKMZSndk7yTUtp928DL2.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjYiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ
Coordinator: Got endpoint ws://localhost:8766 for message to did:peer:2.Vz6MkrW4B8Pyv89nQ3EdN3zCGrLEgEZ5VqsSXjqjhjuwjSzFc.Ez6LSboVcakMcH613b8S6jFgEuA4byKMZSndk7yTUtp928DL2.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjYiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ
Coordinator: Added message to queue for ws://localhost:8766
Successfully sent cohort announcement https://btc1.tools/musig2/keygen/cohort_advert to did:peer:2.Vz6MkrW4B8Pyv89nQ3EdN3zCGrLEgEZ5Vq

# Coordinator checks cohort is set and active

In [30]:
cohort = coordinator.cohorts[0]

In [31]:
if cohort.status != "COHORT_SET":
    print("Cohort not yet set, waiting for beacon participants to join")

# Coordinator waits for signatures to be requested by participants

Or maybe they wait for a certain amount of time. 

In [32]:
cohort.pending_signature_requests

{'did:peer:2.Vz6Mkq8t1GaVjQ5zf8PnnahySB2bHukdVVCUpYQiAf2Uxisxx.Ez6LSqKuutdBvc8zyQkSjJZ7zUBbopj7VDMhasaGLEZYSwsxL.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjciLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ': 'Hello, world!'}

# Coordinator starts signing session

In [33]:
await coordinator.start_signing_session(cohort.id)

Attempting to start signing session for bacfedf0-428d-4b84-bd9e-75b217b93f7c
Cohort bacfedf0-428d-4b84-bd9e-75b217b93f7c found. Starting signing session.
DEBUG: Starting start_signing_session
Starting signing session for cohort bacfedf0-428d-4b84-bd9e-75b217b93f7c with status COHORT_SET
DEBUG: Creating SMT root bytes
DEBUG: Setting up transaction inputs
DEBUG: Creating script pubkey
DEBUG: Creating beacon signal txout
DEBUG: Creating refund output
DEBUG: Creating pending beacon signal
DEBUG: Creating signing session
DEBUG: Cleaning up and returning
Signing session 33356846-6e12-431d-ae02-387a1f5f3894 created for cohort bacfedf0-428d-4b84-bd9e-75b217b93f7c
Starting signing session 33356846-6e12-431d-ae02-387a1f5f3894 for cohort bacfedf0-428d-4b84-bd9e-75b217b93f7c
Sending authorization request to did:peer:2.Vz6Mkq8t1GaVjQ5zf8PnnahySB2bHukdVVCUpYQiAf2Uxisxx.Ez6LSqKuutdBvc8zyQkSjJZ7zUBbopj7VDMhasaGLEZYSwsxL.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjciLCJhIjpbImRpZGNvbW0vdjIiX